In [1]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_loaders import AsyncHtmlLoader

from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain_community.document_transformers import Html2TextTransformer

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
def load_html(url):
    # Load HTML
    loader = AsyncChromiumLoader([url])
    html = loader.load()

    # Transform
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span"])

    # Result
    return docs_transformed[0].page_content[0:500]


In [4]:
def load_html_2(list_urls):
    loader = AsyncHtmlLoader(list_urls)
    docs = loader.load()

    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed[0].page_content[0:500]

In [5]:
urls = ["https://www.nu.nl/economie/6351889/consumentenbond-opent-meldpunt-voor-gedupeerden-variabel-energiecontract.html"]

In [7]:
ans = load_html(urls[0])
print(ans)


USER_AGENT environment variable not set, consider setting it to identify your requests.


RuntimeError: asyncio.run() cannot be called from a running event loop

In [8]:
res = load_html_2(urls)
print(res)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.05it/s]

In [9]:
res

'\n\n'

In [10]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.espn.com", "https://lilianweng.github.io/posts/2023-06-23-agent/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 2/2 [00:00<00:00,  4.33it/s]


In [12]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.llms.ollama import Ollama
from langchain.tools import tool
import requests
from bs4 import BeautifulSoup

# 🧠 Initialize the local Ollama model
llm = Ollama(model="llama3.2:3b")  # Replace with the model you're using, like 'mistral'

# 🛠️ Create a custom tool for scraping
@tool
def scrape_website(url: str) -> str:
    """Scrapes and returns main text content from a given URL."""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract paragraphs and join them
        paragraphs = soup.find_all('p')
        text = "\n".join(p.get_text() for p in paragraphs)
        return text[:5000]  # Keep it under token limit
    except Exception as e:
        return f"Error scraping site: {e}"

# 🧠 Create the agent with the scraping tool
tools = [scrape_website]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ✅ Use the agent
url = "https://www.nu.nl/economie/6351918/boeing-schikt-met-deel-nabestaanden-van-slachtoffers-ethiopian-airlines-crash.html"
scraped_text = agent.run(f"Scrape the website: {url}")

# 📝 Summarize the scraped content
summary_prompt = f"Please summarize the following content:\n\n{scraped_text}"
summary = llm(summary_prompt)

print("\n📌 Summary:")
print(summary)




> Entering new AgentExecutor chain...
Action: scrape_website
Action Input: https://www.nu.nl/economie/6351918/boeing-schikt-met-deel-nabestaanden-van-slachtoffers-ethiopian-airlines-crash.html
Observation: 
Thought:Question: Scrape the website: https://www.nu.nl/economie/6351918/boeing-schikt-met-deel-nabestaanden-van-slachtoffers-ethiopian-airlines-crash.html
Thought: Action: scrape_website
Action Input: https://www.nu.nl/economie/6351918/boeing-schikt-met-deel-nabestaanden-van-slachtoffers-ethiopian-airlines-crash.html
Observation: 
Thought:Question: Scrape the website: https://www.nu.nl/economie/6351918/boeing-schikt-met-deel-nabestaanden-van-slachtoffers-ethiopian-airlines-crash.html
Thought: Action: scrape_website
Action Input: https://www.nu.nl/economie/6351918/boeing-schikt-met-deel-nabestaanden-van-slachtoffers-ethiopian-airlines-crash.html
Observation: 
Thought:Question: Scrape the website: https://www.nu.nl/economie/6351918/boeing-schikt-met-deel-nabestaanden-van-slachtoffe